In [1]:
import pickle
from pathlib import Path
from math import exp, log
import random

risk_threshold = 6

In [2]:
NamedCountries = ["Singapore", "UK", "India", "US"]

In [3]:
# Define the classes

def classes(code):

    high = code[0]

    try:
        low = int(code[1])
    except IndexError:
        low = ""

    if high == "p":
        if low == "":
            return "Political"
        elif low == 1:
            return "War"
        elif low == 2:
            return "Dividedness"
        elif low == 3:
            return "Scandals"
        elif low == 4:
            return "Cultural diplomacy"
        else:
            return "Incorrect code"
    elif high == "e":
        if low == "":
            return "Economic"
        elif low == 1:
            return "Economic instability"
        elif low == 2:
            return "Government policy"
        elif low == 3:
            return "Economic Stagnation"
        else:
            return "Incorrect code"
    else:
        return "Incorrect code"

In [4]:
# Create a node: a class that has the behaviour of a node

class node:
    def __init__(self, country, subclass):
        self.country = country
        self.subclass = subclass

        self.value = 0 # No risk when initialised

        self.events = [] # Store the ids of events that have affected this node

        self.change = 0

        # print(f"Node for {self.country} initialised.")
    
    def update(self, score):
        self.change = score

        score += self.value

        def UpdateFunction(score):
            return - exp(-(score/5 - log(10))) + 10
        
        self.value = UpdateFunction(score)

In [5]:
# Create a class to store bilateral risks

class bilateral:
    def __init__(self, countries):
        self.cause = [] # Refer to an event ID

        self.countries = [] # Countries is a list of length 2 containing class instances of 2 countries

        for country in countries:
            self.countries.append(country.name)

        self.political = node(self.countries, classes("p"))
        self.economic = node(self.countries, classes("e"))

        # print(f"The nodes {self.political.country} and {self.economic.country} have been initialised.")

        # Weight gives a link from the bilateral relations to the weights of the first country
        self.weights = [[0.5 for i in range(7)] for j in range(2)]

        self.loc = "Weights\\" + "".join(self.countries) + ".bin"
        self.path = Path(self.loc)

        if self.path.is_file():
            with open(self.loc, "rb") as f:
                self.weights = pickle.load(f)
        else:
            self.weights = {"domestic": [[0.5 for i in range(7)] for j in range(2)]}
            with open(self.loc, "wb") as f:
                pickle.dump(self.weights, f)

Order of weights:
1. Domestic: 7 x 7 where 1st index is start and 2nd index is end. Order is p1, p2, p3, p4, e1, e2, e3
2. < Country name >: 7 x 9  each index in the 2 d matrix gives a connection and the direction is from this country to the country it is connected to (the other direction is accessible from the other country)
    **The last two in the 9 are the bilateral nodes.**
3. Bilateral comes at the end as a set of 2: order is Political = 0 then Economic = 1

In [6]:
def GetBilateral(country1id, country2id):
    # The input is the ids of two countries

    # The index of country1 in countries is less than the index of country2 (the check is below)

    if country2id < country1id:
        temp = country2id
        country2id = country1id
        country1id = temp

    a = [i for i in range(len(countries))]

    loopno = 0

    for i in range(len(a)):
        for j in range(len(a) - i - 1):
            if [a[i], a[j + i + 1]] == [country1id, country2id]:
                # print(f"The countries selected are {BilateralInfo[loopno].countries}, {BilateralInfo[loopno].political.country}, {BilateralInfo[loopno].economic.country}")
                return BilateralInfo[loopno]
            loopno += 1
            
    raise Exception("This combination of countries is invalid")
    
    # The output is the class instace of the bilateral relations between the countries in the list BilateralInfo

In [7]:
def Update(node, change, weight):
    update = change * weight

    # print(f"The update at {node.country} {node.subclass} is {update}.")

    if update > risk_threshold:
        print(f"There is a grave situation in {node.country} in the field of {node.subclass}. The risk increases by the risk score of {update}")
    
    node.update(update)

In [8]:
# Create a class for the different countries in the graph

class domestic:
    def __init__(self, name):
        self.name = name
        self.id = NamedCountries.index(name)
        self.nodes = []

        # Political
        for i in range(1, 5):
            self.nodes.append(node(self.name, classes("p" + str(i))))

        # Economic
        for i in range(1, 4):
            self.nodes.append(node(self.name, classes("e" + str(i))))

        self.loc = "Weights\\" + self.name + ".bin"
        self.path = Path(self.loc)

        if self.path.is_file():
            with open(self.loc, "rb") as f:
                self.weights = pickle.load(f)
        else:
            self.weights = {"domestic": [[random.random() for i in range(7)] for j in range(7)]}
            with open(self.loc, "wb") as f:
                pickle.dump(self.weights, f)

    def AddLink(self, country):
        # Country is a class instance of the domestic class
        if self.name == country.name:
            return None

        self.weights[country.name] = [[random.random() for i in range(9)] for j in range(7)]
        
        # This function only creates weights in one direction
    
    def FindNode(self, code):
        if code[0] == "p":
            return int(code[1]) - 1
        elif code[0] == "e":
            return int(code[1]) + 4
        else:
            print("Invalid subclass code")
            return None
        
        # Thus function returns the node number in the pre-defined format

    def propagate(self, StartNode):
        # Only changes get propagrated, not actual values
        # Self.change has to be updated before this behaviour is called

        # Start node is the number of the node - an integer from 1 to 7

        # Domestic propagation
        for i in range(7):
            if i != StartNode:
                Update(self.nodes[i], self.nodes[StartNode].change, self.weights["domestic"][StartNode][i])
                
                if self.nodes[i].value > 8:
                    print(f"There is a grave situation in {self.name} in the field of {self.nodes[i].subclass} due to an event {self.nodes[StartNode].events[-1]}.")
        
        # International propagation
        for country in self.weights.keys():
            if country != "domestic":
                CountryID = NamedCountries.index(country) # ID of the international country

                for i in range(7):
                    Update(countries[CountryID].nodes[i], self.nodes[StartNode].change, self.weights[country][StartNode][i])
                
                rel = GetBilateral(CountryID, self.id)
                i = 7
                Update(rel.political, self.nodes[StartNode].change, self.weights[country][StartNode][i])
                rel.cause.append(self.nodes[StartNode].events[-1])
                
                i = 8
                Update(rel.economic, self.nodes[StartNode].change, self.weights[country][StartNode][i])
                rel.cause.append(self.nodes[StartNode].events[-1])

                if ( rel.political.value + rel.economic.value ) / 2 > 8:
                    print(f"There is a grave situation in the relations between {self.name} and {country} due to the event id {self.nodes[StartNode].events[-1]}.")
                    print(rel.countries, ":", ( rel.political.value + rel.economic.value ) / 2 )

In [9]:
countries = []

for name in NamedCountries:
    countries.append(domestic(name))

# Intialise bilateral relations

BilateralInfo = []

for i in range(len(countries)):
    for j in range(len(countries) - i - 1):
        BilateralInfo.append(bilateral([countries[i], countries[j + i + 1]]))

# Initalise links

for i in range(len(countries)):
    for j in range(len(countries)):
        if i != j:
            countries[i].AddLink(countries[j])

The nodes ['Singapore', 'UK'] and ['Singapore', 'UK'] have been initialised.
The nodes ['Singapore', 'India'] and ['Singapore', 'India'] have been initialised.
The nodes ['Singapore', 'US'] and ['Singapore', 'US'] have been initialised.
The nodes ['UK', 'India'] and ['UK', 'India'] have been initialised.
The nodes ['UK', 'US'] and ['UK', 'US'] have been initialised.
The nodes ['India', 'US'] and ['India', 'US'] have been initialised.


In [10]:
BilateralInfo[3].political.country

['UK', 'India']

In [11]:
def ExtractNodes(countries, BilateralInfo):
    # This serves to get the value of all the nodes in the network so that they can be compared for the purpose of training

    NodeList = []

    for country in countries:
        NodeList += country.nodes

    NodeListIntl = []

    for relation in BilateralInfo:
        NodeListIntl += [relation.political, relation.economic]

    return NodeList, NodeListIntl

In [12]:
def ReloadNodes(NodeList, NodeListIntl, countries, BilateralInfo):
    # This takes in the list of nodes and then updates them in all the nodes
    
    for country in countries:
        country.nodes = NodeList[0:len(country.nodes)]

        del NodeList[0:len(country.nodes)]

    if NodeList == []:
        print("Successful reloading")
    else:
        raise Exception("Reload error")
    
    i = 0

    for relation in BilateralInfo:
        relation.political = NodeListIntl[i + 0]
        relation.economic = NodeListIntl[i + 1]
        i += 2

In [13]:
loc = "Weights\\Nodes.bin"

try:
    with open(loc, "rb") as f:
        nodes = pickle.load(f)

    ReloadNodes(nodes[0], nodes[1], countries, BilateralInfo)
    
except: 
    AllNodes = ExtractNodes(countries, BilateralInfo)

    with open(loc, "wb") as f:
        pickle.dump(AllNodes, f)

    print("Nodes freshly initialised in Nodes.bin")

Nodes freshly initialised in Nodes.bin


In [35]:
# Estimate memory requirement for this model



In [36]:
countries[3].weights

{'domestic': [[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]],
 'Singapore': [[0.3890687499367401,
   0.6492668282601061,
   0.9051823934342575,
   0.9919912850656996,
   0.505616889266717,
   0.27521128933249295,
   0.25303711093793646,
   0.3136365348320954,
   0.4264832913049218],
  [0.6285581213706627,
   0.22398527162907744,
   0.2335585605769559,
   0.1475436873070919,
   0.709967373476716,
   0.3256454585908075,
   0.6125303384169436,
   0.30328493005574564,
   0.4314648935839178],
  [0.07273684753194343,
   0.6219329437377794,
   0.314378241888276,
   0.24033922838905586,
   0.11010910388844497,
   0.9548089360623824,
   0.050832238642199545,
   0.6272072806633042,
   0.5456361497995894],
  [0.1511065220228539,
   0.27809670998021707,
   0.9081078726486402,
 

# Process events

In [37]:
def EventProcess(id, country, subclass, score, propagate, TwoLevel):

    """"
    ID gives the ID of the event in the event database
    Country is the country where the event took place
    Subclass is the subclass of the event (in the code)
    Score is the change in the risk score due to that event
    propagate indicates whether the conseuqneces of the events need to be propagated (boolean)
    TwoLevel indicates whether multiple propagations are required (boolean)
    """
    CountryID = NamedCountries.index(country)
    NodeAffected = countries[CountryID].FindNode(subclass)

    countries[CountryID].nodes[NodeAffected].update(score)
    countries[CountryID].nodes[NodeAffected].events.append(id)

    # The next step is to propagate the change to the surroudning nodes

    if propagate:
        countries[CountryID].propagate(NodeAffected)

    # Add the second level propagation here

    if TwoLevel: 
        pass

In [41]:
EventProcess(2, "UK", "p3", 6, True, True)

There is a grave situation in UK in the field of War due to an event 2.
There is a grave situation in UK in the field of Dividedness due to an event 2.
There is a grave situation in UK in the field of Cultural diplomacy due to an event 2.
There is a grave situation in UK in the field of Economic instability due to an event 2.
There is a grave situation in UK in the field of Government policy due to an event 2.
There is a grave situation in UK in the field of Economic Stagnation due to an event 2.
There is a grave situation in the relations between UK and Singapore due to the event id 2.
['Singapore', 'UK'] : 8.155668543390476


# Get risk analysis

In [42]:
country1 = input("Enter the country whose risk situation you want to look at:")

CountryID1 = NamedCountries.index(country1)

country2 = input("Enter the country with respect to which you want to look at the risk situation:")

CountryID2 = NamedCountries.index(country2)

bilateral = GetBilateral(CountryID1, CountryID2)

print(f"The following is the bilateral relations between {bilateral.countries[0]} and {bilateral.countries[1]}")

print(f"""Political risk: {bilateral.political.value}
Economic risk: {bilateral.economic.value}

The reason for this risk are the following events:
{bilateral.cause}
""")

The following is the bilateral relations between Singapore and UK
Political risk: 8.14437553246251
Economic risk: 8.166961554318442

The reason for this risk are the following events:
[1, 1, 2, 2, 2, 2, 2, 2]



# Training

Only in this phase can the weights stored can be modified.

In [39]:
# The code below needs to be executed for each record in the database

InitialNodeSet = ExtractNodes(countries, BilateralInfo)
EventProcess( , False, False) # Pass parameters from the database
PredictedFuture = ExtractNodes(countries, BilateralInfo)

"""
# This is for a consequence lifespan of 1 month
for all the days in the coming month:
    for all the events in the next day:
        EventProcess( , False, False) # Pass parameters from the database
    ActualFuture = ExtractNodes()
    learn(PredictedFuture, AcutalFuture) # Minimise the error between these (conventional ML)
"""

In [ ]:
def learn():
    pass

In [19]:
loc = "Weights\\Nodes.bin"

with open(loc, "wb") as f:
    pickle.dump(nodes, f)

In [20]:
with open(loc, "rb") as f:
    nodes = pickle.load(f)